In [145]:
data_folder = './reuters21578/'

sgml_number_of_files = 22
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

# Word2Vec number of features
num_features = 500
# Limit each newsline to a fixed number of words
# document_max_num_words = 100
# Selected categories
# selected_categories = ['pl_usa']

In [146]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [147]:
# Create category dataframe

# Read all categories
category_data = []

for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
news_categories = pd.DataFrame(data=category_data)

# print "category_data: ", category_data
news_categories

,0,1
0,to_acq,Topics
1,to_alum,Topics
2,to_austdlr,Topics
3,to_austral,Topics
4,to_barley,Topics
5,to_bfr,Topics
6,to_bop,Topics
7,to_can,Topics
8,to_carcass,Topics
9,to_castor-meal,Topics


In [148]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

In [149]:
def to_category_vector(categories):
    vector = zeros(len(categories)).astype(float32)
    
    for i in range(len(categories)):
        if target_categories[i] in categories:
            vector[i] = 1.0
    
    return vector

In [150]:
lemmatizer = WordNetLemmatizer()
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
stop_words = set(stopwords.words("english"))

def cleanUpSentence(r, stop_words = None):
    r = r.lower().replace("<br />", " ")
    r = re.sub(strip_special_chars, "", r.lower())
    if stop_words is not None:
        words = word_tokenize(r)
        filtered_sentence = []
        for w in words:
            w = lemmatizer.lemmatize(w)
            if w not in stop_words:
                filtered_sentence.append(w)
        return " ".join(filtered_sentence)
    else:
        return r

In [151]:
# Parse SGML files
document_X = {}
document_Y = {}
docid_traintest = {}
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)

# Iterate all files
# for i in range(sgml_number_of_files):
#     if i < 10:
#         seq = '00' + str(i)
#     else:
#         seq = '0' + str(i)
        
#     file_name = sgml_file_name_template.replace('NNN', seq)
#     print('Reading file: %s' % file_name)
    #data_folder + file_name
with open(data_folder+'reut2-000.sgm', 'rb') as file:
    
    content = BeautifulSoup(file.read().lower(),'html.parser')

    for newsline in content('reuters'):
        document_categories = []

        # News-line Id
        document_id = newsline['newid']
#             print document_id,
        train_test = newsline['lewissplit']
        docid_traintest[document_id] = train_test
#             print "train_test: ",train_test

        # News-line text
        document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
        doc_categories=strip_tags(str(newsline('topics')[0].body))
        doc_categories = unescape(doc_categories)

        document_body = unescape(document_body)

        # News-line categories
        topics = newsline.topics.contents
        places = newsline.places.contents
        people = newsline.people.contents
        orgs = newsline.orgs.contents
        exchanges = newsline.exchanges.contents

        for topic in topics:
            document_categories.append('to_' + strip_tags(str(topic)))

        for place in places:
            document_categories.append('pl_' + strip_tags(str(place)))

        for person in people:
            document_categories.append('pe_' + strip_tags(str(person)))

        for org in orgs:
            document_categories.append('or_' + strip_tags(str(org)))

        for exchange in exchanges:
            document_categories.append('ex_' + strip_tags(str(exchange)))
#             print "document_categories: ",document_categories
        # Create new document    
#             update_frequencies(document_categories)

        document_X[document_id] = document_body
        document_Y[document_id] = document_categories
#print(document_Y)

In [152]:
# data preprocessing

In [153]:
totalX = []
#totalY = np.array(document_Y)
nltk.download('punkt')
nltk.download('wordnet')
#print(document_X.keys())
for i, doc in document_X.items():
    #print(i)
    totalX.append(cleanUpSentence(doc, stop_words))


print(totalX[0])

[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


shower continued throughout week inthe bahia cocoa zone alleviating drought since earlyjanuary improving prospect coming temporaoalthough normal humidity level restoredcomissaria smith said weekly review dry period mean temporao late year arrival week ended february 22 155221 bagsof 60 kilo making cumulative total season 593mln 581 stage last year seemsthat cocoa delivered earlier consignment wa included thearrivals figure comissaria smith said still doubt howmuch old crop cocoa still available harvesting haspractically come end total bahia crop estimatesaround 64 mln bag sale standing almost 62 mln thereare hundred thousand bag still hand farmersmiddlemen exporter processor doubt much cocoa would fitfor export shipper experiencing dificulties inobtaining bahia superior certificate view lower quality recent week farmer havesold good part cocoa held consignment comissaria smith said spot bean price rose 340 350cruzados per arroba 15 kilo bean shipper reluctant offer nearby shipment ando

In [154]:
import numpy as np
max_vocab_size = 200
input_tokenizer = Tokenizer(200)# change accuracy....
input_tokenizer.fit_on_texts(totalX)
#print(input_tokenizer.word_counts)
encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
print(encoded_docs.shape)
input_vocab_size = len(input_tokenizer.word_index) + 1
print("input_vocab_size:",input_vocab_size)
totalX = np.array(pad_sequences(input_tokenizer.texts_to_sequences(totalX)))
# print(input_tokenizer.word_counts)
# print(t.document_count)
# print(t.word_index)
# print(t.word_docs)
# print(totalX.counts)

(1000, 200)
input_vocab_size: 17601


In [155]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_questions.Tags)
y = multilabel_binarizer.classes

NameError: name 'df_questions' is not defined